In [ ]:
#Dostęp do dysku
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [169]:
#Biblioteki
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
from sklearn.model_selection import train_test_split
import gensim
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import TaggedDocument
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
#Wczytanie dnaych do trenowania
df = pd.read_csv('/content/drive/MyDrive/result.csv', sep='\t')
df = df[['Category','Title','RawWiki','CleanWiki']]
df.head(45)

,Category,Title,RawWiki,CleanWiki
0,Animals,Tiger,The tiger (Panthera tigris) is the largest ext...,the tiger panthera tigris is the largest extan...
1,Animals,Rabbit,Rabbits are small mammals in the family Lepori...,rabbits are small mammals in the family lepori...
2,Animals,Koala Bear,"The koala or, inaccurately, koala bear[a] (Ph...",the koala or inaccurately koala bear a phascol...
3,Animals,Guinea Pig,The guinea pig or domestic guinea pig (Cavia ...,the guinea pig or domestic guinea pig cavia po...
4,Animals,Mouse,"A mouse, plural mice, is a small rodent. Chara...",a mouse plural mice is a small rodent characte...
5,Animals,Penguin,Penguins (order Sphenisciformes /sfɪˈnɪsɪfɔːrm...,penguins order sphenisciformes sfɪˈnɪsɪfɔːrmiː...
6,Animals,Cat,The cat (Felis catus) is a domestic species of...,the cat felis catus is a domestic species of s...
7,Animals,Dog,The dog (Canis familiaris when considered a di...,the dog canis familiaris when considered a dis...
8,Animals,Lion,The lion (Panthera leo) is a species in the fa...,the lion panthera leo is a species in the fami...
9,Animals,Fox,"Foxes are small to medium-sized, omnivorous ma...",foxes are small to medium sized omnivorous mam...


In [ ]:
df.shape

(45, 4)

In [58]:
df['CleanWiki'].apply(lambda x: len(x.split(' '))).sum()

14884

In [111]:
train['CleanWiki']

17    field hockey is a widely played team sport of ...
13    the giraffe giraffa is an african artiodactyl ...
42    the trombone is a musical instrument in the br...
15    football is a family of team sports that invol...
9     foxes are small to medium sized omnivorous mam...
16    badminton is a racquet sport played using racq...
37    the clarinet is a family of woodwind instrumen...
31    the piano is an acoustic stringed musical inst...
27    skiing is a means of transport using skis to g...
0     the tiger panthera tigris is the largest extan...
30    the guitar is a fretted musical instrument tha...
29    rugby football is a collective name for the fa...
5     penguins order sphenisciformes sfɪˈnɪsɪfɔːrmiː...
11    zebras a subgenus hippotigris are african equi...
33    a drum kit also called a drum set trap set an ...
1     rabbits are small mammals in the family lepori...
40    the harp is a stringed musical instrument that...
21    cricket is a bat and ball game played betw

In [54]:
print(len(df['CleanWiki']))

45


In [63]:
print(df['CleanWiki'][1])

rabbits are small mammals in the family leporidae of the order lagomorpha along with the hare and the pika oryctolagus cuniculus includes the european rabbit species and its descendants the world s breeds of domestic rabbit sylvilagus includes wild rabbit species among them the seven types of cottontail the european rabbit which has been introduced on every continent except antarctica is familiar throughout the world as a wild prey animal and as a domesticated form of livestock and pet with its widespread effect on ecologies and cultures the rabbit or bunny is in many areas of the world a part of daily life as food clothing a companion and a source of artistic inspiration although once considered rodents lagomorphs like rabbits have been discovered to have diverged separately and earlier than their rodent cousins and have a number of traits rodents lack like two extra incisors


In [103]:
def tokenize_text(text):
  tokens = []
  for sent in nltk.sent_tokenize(df['CleanWiki'][text]):
  #for sent in nltk.sent_tokenize(text):
    #print(sent)
    for word in nltk.word_tokenize(sent):
      #print(word)
      if len(word) < 2:
        continue
      tokens.append(word.lower())
  #print(tokens)
  return tokens

In [113]:
print(test['CleanWiki'])

39    the pipe organ is a musical instrument that pr...
25    swimming is the self propulsion of a person th...
26    handball also known as team handball european ...
43    the viola viˈoʊlə vee oh lə also uk vaɪˈoʊlə v...
35    the cello ˈtʃɛloʊ chel oh plural celli or cell...
41    the ukulele ˌjuːkəˈleɪli yoo kə lay lee from h...
4     a mouse plural mice is a small rodent characte...
12    the donkey or ass equus africanus asinus is a ...
8     the lion panthera leo is a species in the fami...
3     the guinea pig or domestic guinea pig cavia po...
6     the cat felis catus is a domestic species of s...
24    golf is a club and ball sport in which players...
32    the violin sometimes known as a fiddle is a wo...
19    basketball colloquially referred to as hoops i...
Name: CleanWiki, dtype: object


In [121]:
tokens=[]
for i in range(len(df['CleanWiki'])):
  toke = tokenize_text(i)
  tokens.append(toke)

print(tokens)

[['the', 'tiger', 'panthera', 'tigris', 'is', 'the', 'largest', 'extant', 'cat', 'species', 'and', 'member', 'of', 'the', 'genus', 'panthera', 'it', 'is', 'most', 'recognisable', 'for', 'its', 'dark', 'vertical', 'stripes', 'on', 'orange', 'brown', 'fur', 'with', 'lighter', 'underside', 'it', 'is', 'an', 'apex', 'predator', 'primarily', 'preying', 'on', 'ungulates', 'such', 'as', 'deer', 'and', 'wild', 'boar', 'it', 'is', 'territorial', 'and', 'generally', 'solitary', 'but', 'social', 'predator', 'requiring', 'large', 'contiguous', 'areas', 'of', 'habitat', 'which', 'support', 'its', 'requirements', 'for', 'prey', 'and', 'rearing', 'of', 'its', 'offspring', 'tiger', 'cubs', 'stay', 'with', 'their', 'mother', 'for', 'about', 'two', 'years', 'before', 'they', 'become', 'independent', 'and', 'leave', 'their', 'mother', 'home', 'range', 'to', 'establish', 'their', 'own', 'the', 'tiger', 'once', 'ranged', 'widely', 'from', 'the', 'eastern', 'anatolia', 'region', 'in', 'the', 'west', 'to', '

In [106]:
documents = [TaggedDocument(doc, tags=[df['Category'][i]]) for i, doc in enumerate(tokens)]
print(documents[0])

TaggedDocument(['the', 'tiger', 'panthera', 'tigris', 'is', 'the', 'largest', 'extant', 'cat', 'species', 'and', 'member', 'of', 'the', 'genus', 'panthera', 'it', 'is', 'most', 'recognisable', 'for', 'its', 'dark', 'vertical', 'stripes', 'on', 'orange', 'brown', 'fur', 'with', 'lighter', 'underside', 'it', 'is', 'an', 'apex', 'predator', 'primarily', 'preying', 'on', 'ungulates', 'such', 'as', 'deer', 'and', 'wild', 'boar', 'it', 'is', 'territorial', 'and', 'generally', 'solitary', 'but', 'social', 'predator', 'requiring', 'large', 'contiguous', 'areas', 'of', 'habitat', 'which', 'support', 'its', 'requirements', 'for', 'prey', 'and', 'rearing', 'of', 'its', 'offspring', 'tiger', 'cubs', 'stay', 'with', 'their', 'mother', 'for', 'about', 'two', 'years', 'before', 'they', 'become', 'independent', 'and', 'leave', 'their', 'mother', 'home', 'range', 'to', 'establish', 'their', 'own', 'the', 'tiger', 'once', 'ranged', 'widely', 'from', 'the', 'eastern', 'anatolia', 'region', 'in', 'the', '

In [205]:
all = documents

In [206]:
train_tagged, test_tagged = train_test_split(all, test_size=0.3, random_state=42)
all=train_tagged

In [126]:
#DBOW Distributed Bag of Words dm=0
#The paragraph vectors are obtained 
#by training a neural network on the task of predicting a probability distribution of words in a paragraph given a randomly-sampled word from the paragraph.
#DM Distributed Memory dm=1
#    If dm=0, distributed bag of words (PV-DBOW) is used; if dm=1,‘distributed memory’ (PV-DM) is used.
   # 300- dimensional feature vectors.
   # min_count=2, ignores all words with total frequency lower than this.
   # negative=5 , specifies how many “noise words” should be drawn.
   #hs=0 , and negative is non-zero, negative sampling will be used.
   #3 sample=0 , the threshold for configuring which higher-frequency words are randomly down sampled.
   # workers=cores , use these many worker threads to train the model (=faster training with multicore machines).

In [207]:
import multiprocessing
cores = multiprocessing.cpu_count()

In [208]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, regressors

In [209]:
model1 = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores)
model2 = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model3 = Doc2Vec(dm=1, dm_mean=1, vector_size=300, window=10, negative=5, min_count=1, workers=5, alpha=0.065, min_alpha=0.065)

In [210]:
model1.build_vocab(all)
model2.build_vocab(all)
model3.build_vocab(all)

In [211]:
%%time
for epoch in range(30):
    model1.train(utils.shuffle([x for x in tqdm(all)]), total_examples=len(all), epochs=1)
    model1.alpha -= 0.002
    model1.min_alpha = model1.alpha

100%|██████████| 31/31 [00:00<00:00, 84266.64it/s]


CPU times: user 1.11 s, sys: 72.9 ms, total: 1.18 s
Wall time: 1.51 s


In [212]:
y_train, X_train = vec_for_learning(model1, train_tagged)
y_test, X_test = vec_for_learning(model1, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

Testing accuracy 0.5
Testing F1 score: 0.5003607503607503


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [214]:
model1.save('/content/drive/MyDrive/d2v_model_dbow.doc2vec')

In [216]:
model1.save('/content/drive/MyDrive/d2v_model_dbow.pkl')

In [218]:
for epoch in range(30):
    model2.train(utils.shuffle([x for x in tqdm(all)]), total_examples=len(all), epochs=1)
    model2.alpha -= 0.002
    model2.min_alpha = model2.alpha

y_train, X_train = vec_for_learning(model2, train_tagged)
y_test, X_test = vec_for_learning(model2, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

100%|██████████| 31/31 [00:00<00:00, 137590.92it/s]


Testing accuracy 0.5
Testing F1 score: 0.460482374768089


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [220]:
for epoch in range(30):
    model3.train(utils.shuffle([x for x in tqdm(all)]), total_examples=len(all), epochs=1)
    model3.alpha -= 0.002
    model3.min_alpha = model3.alpha

y_train, X_train = vec_for_learning(model3, train_tagged)
y_test, X_test = vec_for_learning(model3, test_tagged)
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print('Testing accuracy %s' % accuracy_score(y_test, y_pred))
print('Testing F1 score: {}'.format(f1_score(y_test, y_pred, average='weighted')))

100%|██████████| 31/31 [00:00<00:00, 148598.20it/s]


Testing accuracy 0.9285714285714286
Testing F1 score: 0.9267161410018553


In [221]:
model3.save('/content/drive/MyDrive/best92.pkl')